In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
df.head()

In [ ]:
import nltk
#convert to lower-case
def converttolowercase(text):
    return text.lower();
df['review']=df['review'].apply(converttolowercase)
df.head()

In [ ]:
#removing html tags
import re
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)
df['review']=df['review'].apply(remove_tags)
df.head()

In [ ]:
#removing stopwords
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.tokenize import word_tokenize

def remove_stopwords(text):
    stop_words = set(stopwords.words("english")) 
    tokens=word_tokenize(text)
    tokens_without_sw = [word for word in tokens if not word in stop_words]
    filtered_text = ' '.join(tokens_without_sw)
    return filtered_text
df['review']=df['review'].apply(remove_stopwords)
df.head()

In [ ]:
#removing punctuation
import string
def remove_punctuations(text):
    translator = str.maketrans('', '', string.punctuation) 
    return text.translate(translator) 
df['review']=df['review'].apply(remove_punctuations)
df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(ngram_range = (2,2),tokenizer = token.tokenize)  #using a bi-gram model
text_counts = cv.fit_transform(df['review'])
text_counts

In [ ]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(text_counts, df['sentiment'], test_size=0.20, random_state=42)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

In [ ]:
MNB = MultinomialNB()
MNB.fit(X_train1, Y_train1)

In [ ]:
from sklearn import metrics
predicted = MNB.predict(X_test1)
accuracy_score = metrics.accuracy_score(predicted, Y_test1)
print(accuracy_score*100)

In [ ]:
#using lstm
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

In [ ]:
tokenizers = Tokenizer(num_words=5000, split=' ')
tokenizers.fit_on_texts(df['review'].values)
X = tokenizers.texts_to_sequences(df['review'])
X = pad_sequences(X, padding='post', maxlen=100)
print(X[1])

In [ ]:
from keras.layers import Bidirectional
from keras.layers import Dropout
embed_dim = 128
lstm_out = 200
model = Sequential()

model.add(Embedding(5000, embed_dim,input_length = X.shape[1]))

model.add(Bidirectional(LSTM(128,dropout=0.2, recurrent_dropout=0.2)))
#model.add(Dense(512,activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [ ]:
from sklearn.preprocessing import LabelEncoder

Le = LabelEncoder()

y = Le.fit_transform(df['sentiment'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, random_state = 42)

model.fit(X_train, y_train, batch_size =128, epochs = 2)

In [ ]:
predictions = model.predict(X_test)
t=[]
for p in predictions:
    
    if(p>=0.5):
        
        t.append(1)
    else:
        t.append(0)
    
model.evaluate(X_test,y_test)    

In [ ]:
predicted1=MNB.predict(X_test1)


In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print("For Naive Bayes")
print(classification_report(Y_test1, predicted1, target_names = ['Bad Reviews','Good Reviews']))
print(confusion_matrix(Y_test1, predicted1))

In [ ]:
print("For lstm") 
print(classification_report(y_test, t, target_names = ['Bad Reviews','Good Reviews']))
print(confusion_matrix(y_test, t))

In [ ]:
def predictlstm(text):
    tokenizers = Tokenizer(num_words=5000, split=' ')
    tokenizers.fit_on_texts(df['review'].values)
    X = tokenizers.texts_to_sequences(text)
    X = pad_sequences(X, padding='post', maxlen=100)
    l=model.predict(X)
    if(l>=0.5):
        print(['positive'])
    else:
        print(['negative'])
   
  
    

In [ ]:
def predictnb(text):
    t=cv.transform(text)
    nb=MNB.predict(t)
    print(nb)
    
    

In [ ]:
#lstm fails to see the negative sentiment in this review
text1=["This film tried to be too many things all at once: stinging political satire, Hollywood blockbuster, sappy romantic comedy, family values promo... the list goes on and on. It failed miserably at all of them, but there was enough interest to keep me from turning it off until the end.Although I appreciate the spirit behind WAR, INC., it depresses me to see such a clumsy effort, especially when it will be taken by its targets to reflect the lack of the existence of a serious critique, rather than simply the poor writing, direction, and production of this particular film.There is a critique to be made about the corporatization of war. But poking fun at it in this way diminishes the true atrocity of what is happening. Reminds me a bit of THREE KINGS, which similarly trivializes a genuine cause for concern."]
predictlstm(text1)
predictnb(text1)

In [ ]:
#positive sentiment
pos=["The first film, a frenetic and hilarious tribute to superhero films and imbued with 1950s style, was an absolute masterpiece, with great characters and hugely inventive action set pieces. This one is a worthy successor. That’s all you need to know."]
predictlstm(pos)
predictnb(pos)

In [ ]:
#negative sentiment
neg=["A very boring and flop movie."]
predictlstm(neg)
predictnb(neg)

In [ ]:
#somewhat hard to find out the sentiment of the review
t1=["I do not dislike this movie,but I won't recommend it to anyone."]
predictlstm(t1)
predictnb(t1)

In [ ]:
#use of a negative word to describe something positive
t2=["The movie is surprising with plenty of unsettling plot twists."]
predictlstm(t2)
predictnb(t2)